<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#LGBM-training" data-toc-modified-id="LGBM-training-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>LGBM training</a></span><ul class="toc-item"><li><span><a href="#Prepare" data-toc-modified-id="Prepare-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Prepare</a></span></li><li><span><a href="#Core" data-toc-modified-id="Core-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Core</a></span></li><li><span><a href="#Feature-importances" data-toc-modified-id="Feature-importances-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Feature importances</a></span></li></ul></li></ul></div>

# Load

In [1]:
from apps.lib import Sensor, Performance, Model
from apps.score import ScoreBoard
import apps.tkconfig as tkconfig
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb

SONG = 1
WHO = 7
TRAIN_ORDER = 3

Using TensorFlow backend.


In [2]:
sensor_data = Sensor(verbose=0)

In [3]:
scoreboard = ScoreBoard(sensor_data.drummer_df)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
def gogo(who_id, song_id, train_pf_order):
    train_pf = Performance(sensor_data, who_id, song_id, train_pf_order, 4, 2)
    
    pfs = []
    for pf_order in tqdm(range(1, 3 + 1)):
        if pf_order == train_pf_order:
            pfs.append(train_pf)
        else:
            pfs.append(Performance(sensor_data, who_id, song_id, pf_order, 4, 2)) 
            
    return pfs

In [5]:
pfs = []
for drumer in range(1, 8 + 1):
    ppf = gogo(drumer, SONG, TRAIN_ORDER)
    pfs.append(ppf)

100%|██████████| 3/3 [00:34<00:00, 11.65s/it]


In [6]:
def soso(who_id, song_id):
    sfs = []
    for pf_order in range(1, 3 + 1):
        img_scores, _ = scoreboard.get_scores_with_timestamps(who_id, song_id, pf_order)
        sfs.append(img_scores[-1])
    return sfs

In [7]:
sfs = []
for drumer in range(1, 8 + 1):
    sfs.append(soso(drumer, SONG))

3808/3900 [============================>.] - ETA: 0s

In [8]:
pfs[0][2].event_primitive_df

,hit_type,L_AAI,L_AVI,L_ASMA,L_GAI,L_GVI,L_GSMA,L_AAE,L_ARE,L_MAMI,...,R_AZCR,R_GZCR,R_AMCR,R_GMCR,R_AXYCORR,R_AYZCORR,R_AZXCORR,R_GXYCORR,R_GYZCORR,R_GZXCORR
0,1,0.144363,0.000984,0.182714,7.837218,16.855832,11.142000,0.021825,78.277812,0.126178,...,0.428571,0.571429,0.285714,0.571429,-0.544855,-0.684912,0.121344,0.752108,0.457800,-0.055694
1,0,0.172753,0.005393,0.228000,31.572826,161.915005,47.817571,0.035236,1158.758338,0.183308,...,0.142857,0.285714,0.142857,0.285714,-0.185425,-0.418489,0.697969,-0.488330,0.925166,-0.747645
2,0,0.152061,0.000366,0.226429,62.496385,28.197039,79.462143,0.023488,3933.995127,0.156170,...,0.750000,0.250000,0.750000,0.250000,-0.786548,0.911355,-0.795170,0.975555,-0.810204,-0.866550
3,0,0.431467,0.090386,0.643800,56.532036,555.794405,83.879000,0.276550,3751.665457,0.233283,...,0.333333,0.333333,0.333333,0.333333,0.835047,-0.485918,0.075094,-0.244019,0.141725,-0.994565
4,1,0.390009,0.014918,0.629600,40.664224,350.779650,63.690200,0.167024,2004.358734,0.346880,...,0.375000,0.375000,0.375000,0.375000,0.016678,-0.060541,-0.043862,-0.886426,0.493714,-0.532830
5,0,0.201691,0.003187,0.323400,42.791711,18.822723,72.489400,0.043866,1849.953245,0.185637,...,0.285714,0.285714,0.142857,0.285714,0.487971,0.100405,-0.038495,0.689536,-0.754955,-0.864607
6,0,0.127174,0.000095,0.206000,14.423266,37.821719,23.181800,0.016269,245.852329,0.124161,...,0.375000,0.125000,0.375000,0.125000,-0.660409,0.870875,-0.813031,-0.115209,-0.012770,0.833729
7,0,0.095510,0.000210,0.133600,11.464270,6.423106,17.134200,0.009332,137.852601,0.087926,...,0.500000,0.333333,0.166667,0.333333,-0.865026,-0.689175,0.676779,-0.055290,0.310975,0.007419
8,1,0.152996,0.000584,0.222400,18.565111,3.307427,27.880600,0.023992,347.970792,0.149114,...,0.285714,0.571429,0.285714,0.571429,-0.669668,-0.409309,0.522083,-0.256624,-0.473077,0.119202
9,0,0.328453,0.018054,0.498600,53.995201,1544.527742,80.151000,0.125935,4460.009459,0.299490,...,0.428571,0.142857,0.428571,0.142857,0.170319,0.921414,0.224085,-0.725166,0.809888,-0.624116


# LGBM training

## Prepare

In [9]:
def prepare(train_who, train_porder, valid_porder):
    train_df = pfs[train_who - 1][train_porder - 1].event_primitive_df
    x = train_df.drop(['hit_type'], axis=1)
    y = train_df['hit_type']
    
    valid_df = pfs[train_who - 1][valid_porder - 1].event_primitive_df
    x_valid = valid_df.drop(['hit_type'], axis=1)
    y_valid = valid_df['hit_type']
    
    return x, y, x_valid, y_valid

In [10]:
params = dict({
    'learning_rate': 0.5,
    'application': 'multiclass',
    'num_classes': 3,
#     'min_data_in_leaf': 5,
#     'max_depth': 8,
    'num_leaves': 2 ** 3,
    'verbosity': 0,
    'metric': 'multi_error'
})

grid_params = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [8, 10],
}

In [11]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    return cm

In [12]:
def get_multi_error(y, pred_y):
    cnf_matrix = confusion_matrix(y, pred_y)
    
    # Plot normalized confusion matrix
#     plt.figure(figsize=(10, 10))
    cm = plot_confusion_matrix(cnf_matrix, classes=[0, 1, 2], normalize=True,
                          title='Normalized confusion matrix')
    return sum([1 - cm[i][i] for i in range(cm.shape[0])]) / cm.shape[0]

## Core

In [15]:
result_df = pd.DataFrame(columns=['train_who', 'train_porder', 'train_score', 'valid_who', 'valid_porder', 'valid_score', 'test_who', 'test_porder', 'test_score', 'multi_error'])
id_ = 0

for train_who in range(1, 8 + 1):
    for train_porder in range(1, 3 + 1):
        for valid_porder in range(1, 3 + 1):
            if train_porder == valid_porder:
                continue
            x, y, x_valid, y_valid = prepare(train_who, train_porder, valid_porder)
            
            train_set = lgb.Dataset(x, y)
            valid_set = lgb.Dataset(x_valid, y_valid, free_raw_data=False)
            
            watchlist = [valid_set]
            
            model = lgb.train(params, train_set=train_set, valid_sets=watchlist, num_boost_round=200, verbose_eval=5, early_stopping_rounds=100)
            
            for test_who in range(1, 8 + 1):
                for test_porder in range(1, 3 + 1):
                    if test_who == train_who and (test_porder == train_porder or test_porder == valid_porder):
                        continue
                
                    x_test = pfs[test_who - 1][test_porder - 1].event_primitive_df.drop(['hit_type'], axis=1)
                    y_true = pfs[test_who - 1][test_porder - 1].event_primitive_df['hit_type']
                    
                    y_pred = model.predict(x_test, num_iteration=model.best_iteration)
                    predictions = []
                    for xx in y_pred:
                        predictions.append(np.argmax(xx))
                    y_test = pd.Series(data=predictions)
                    
                    multierror = '%.4f' % get_multi_error(y_true, y_test)
                    
#                     train_str = '%d,%d' % (train_who, train_porder)
                    train_score = sfs[train_who - 1][train_porder - 1]
                    
#                     valid_str = '%d,%d' % (train_who, valid_porder)
                    valid_score = sfs[train_who - 1][valid_porder - 1]
                                                   
#                     test_str = '%d,%d' % (test_who, test_porder)
                    test_score = sfs[test_who - 1][test_porder - 1]
                    
                    result_df.loc[id_] = [train_who, train_porder, train_score, train_who, valid_porder, valid_score, test_who, test_porder, test_score, multierror]
                    
                    id_ += 1

Training until validation scores don't improve for 100 rounds.
[5]	valid_0's multi_error: 0.127424
[10]	valid_0's multi_error: 0.130194
[15]	valid_0's multi_error: 0.124654
[20]	valid_0's multi_error: 0.124654
[25]	valid_0's multi_error: 0.121884
[30]	valid_0's multi_error: 0.108033
[35]	valid_0's multi_error: 0.108033
[40]	valid_0's multi_error: 0.113573
[45]	valid_0's multi_error: 0.113573
[50]	valid_0's multi_error: 0.105263
[55]	valid_0's multi_error: 0.105263
[60]	valid_0's multi_error: 0.105263
[65]	valid_0's multi_error: 0.105263
[70]	valid_0's multi_error: 0.102493
[75]	valid_0's multi_error: 0.102493
[80]	valid_0's multi_error: 0.102493
[85]	valid_0's multi_error: 0.102493
[90]	valid_0's multi_error: 0.102493
[95]	valid_0's multi_error: 0.102493
[100]	valid_0's multi_error: 0.102493
[105]	valid_0's multi_error: 0.102493
[110]	valid_0's multi_error: 0.102493
[115]	valid_0's multi_error: 0.102493
[120]	valid_0's multi_error: 0.102493
[125]	valid_0's multi_error: 0.102493
[130]	v

Training until validation scores don't improve for 100 rounds.
[5]	valid_0's multi_error: 0.138504
[10]	valid_0's multi_error: 0.163435
[15]	valid_0's multi_error: 0.163435
[20]	valid_0's multi_error: 0.160665
[25]	valid_0's multi_error: 0.160665
[30]	valid_0's multi_error: 0.163435
[35]	valid_0's multi_error: 0.160665
[40]	valid_0's multi_error: 0.157895
[45]	valid_0's multi_error: 0.157895
[50]	valid_0's multi_error: 0.155125
[55]	valid_0's multi_error: 0.160665
[60]	valid_0's multi_error: 0.157895
[65]	valid_0's multi_error: 0.157895
[70]	valid_0's multi_error: 0.157895
[75]	valid_0's multi_error: 0.157895
[80]	valid_0's multi_error: 0.157895
[85]	valid_0's multi_error: 0.160665
[90]	valid_0's multi_error: 0.160665
[95]	valid_0's multi_error: 0.160665
[100]	valid_0's multi_error: 0.157895
Early stopping, best iteration is:
[4]	valid_0's multi_error: 0.113573
Training until validation scores don't improve for 100 rounds.
[5]	valid_0's multi_error: 0.174515
[10]	valid_0's multi_error:

Training until validation scores don't improve for 100 rounds.
[5]	valid_0's multi_error: 0.124654
[10]	valid_0's multi_error: 0.127424
[15]	valid_0's multi_error: 0.127424
[20]	valid_0's multi_error: 0.113573
[25]	valid_0's multi_error: 0.116343
[30]	valid_0's multi_error: 0.116343
[35]	valid_0's multi_error: 0.119114
[40]	valid_0's multi_error: 0.116343
[45]	valid_0's multi_error: 0.116343
[50]	valid_0's multi_error: 0.110803
[55]	valid_0's multi_error: 0.105263
[60]	valid_0's multi_error: 0.110803
[65]	valid_0's multi_error: 0.110803
[70]	valid_0's multi_error: 0.110803
[75]	valid_0's multi_error: 0.113573
[80]	valid_0's multi_error: 0.110803
[85]	valid_0's multi_error: 0.113573
[90]	valid_0's multi_error: 0.110803
[95]	valid_0's multi_error: 0.108033
[100]	valid_0's multi_error: 0.110803
[105]	valid_0's multi_error: 0.110803
[110]	valid_0's multi_error: 0.113573
[115]	valid_0's multi_error: 0.110803
[120]	valid_0's multi_error: 0.110803
[125]	valid_0's multi_error: 0.113573
[130]	v

Training until validation scores don't improve for 100 rounds.
[5]	valid_0's multi_error: 0.246537
[10]	valid_0's multi_error: 0.238227
[15]	valid_0's multi_error: 0.232687
[20]	valid_0's multi_error: 0.238227
[25]	valid_0's multi_error: 0.232687
[30]	valid_0's multi_error: 0.229917
[35]	valid_0's multi_error: 0.235457
[40]	valid_0's multi_error: 0.229917
[45]	valid_0's multi_error: 0.227147
[50]	valid_0's multi_error: 0.229917
[55]	valid_0's multi_error: 0.227147
[60]	valid_0's multi_error: 0.227147
[65]	valid_0's multi_error: 0.221607
[70]	valid_0's multi_error: 0.221607
[75]	valid_0's multi_error: 0.221607
[80]	valid_0's multi_error: 0.221607
[85]	valid_0's multi_error: 0.221607
[90]	valid_0's multi_error: 0.221607
[95]	valid_0's multi_error: 0.221607
[100]	valid_0's multi_error: 0.221607
[105]	valid_0's multi_error: 0.221607
[110]	valid_0's multi_error: 0.221607
[115]	valid_0's multi_error: 0.221607
[120]	valid_0's multi_error: 0.221607
[125]	valid_0's multi_error: 0.221607
[130]	v

Training until validation scores don't improve for 100 rounds.
[5]	valid_0's multi_error: 0.132964
[10]	valid_0's multi_error: 0.121884
[15]	valid_0's multi_error: 0.121884
[20]	valid_0's multi_error: 0.127424
[25]	valid_0's multi_error: 0.113573
[30]	valid_0's multi_error: 0.119114
[35]	valid_0's multi_error: 0.121884
[40]	valid_0's multi_error: 0.116343
[45]	valid_0's multi_error: 0.110803
[50]	valid_0's multi_error: 0.113573
[55]	valid_0's multi_error: 0.119114
[60]	valid_0's multi_error: 0.113573
[65]	valid_0's multi_error: 0.113573
[70]	valid_0's multi_error: 0.110803
[75]	valid_0's multi_error: 0.110803
[80]	valid_0's multi_error: 0.110803
[85]	valid_0's multi_error: 0.110803
[90]	valid_0's multi_error: 0.113573
[95]	valid_0's multi_error: 0.113573
[100]	valid_0's multi_error: 0.113573
[105]	valid_0's multi_error: 0.113573
[110]	valid_0's multi_error: 0.116343
[115]	valid_0's multi_error: 0.113573
[120]	valid_0's multi_error: 0.113573
[125]	valid_0's multi_error: 0.113573
[130]	v

Training until validation scores don't improve for 100 rounds.
[5]	valid_0's multi_error: 0.130194
[10]	valid_0's multi_error: 0.108033
[15]	valid_0's multi_error: 0.110803
[20]	valid_0's multi_error: 0.110803
[25]	valid_0's multi_error: 0.110803
[30]	valid_0's multi_error: 0.121884
[35]	valid_0's multi_error: 0.113573
[40]	valid_0's multi_error: 0.130194
[45]	valid_0's multi_error: 0.124654
[50]	valid_0's multi_error: 0.124654
[55]	valid_0's multi_error: 0.124654
[60]	valid_0's multi_error: 0.127424
[65]	valid_0's multi_error: 0.127424
[70]	valid_0's multi_error: 0.127424
[75]	valid_0's multi_error: 0.124654
[80]	valid_0's multi_error: 0.124654
[85]	valid_0's multi_error: 0.124654
[90]	valid_0's multi_error: 0.124654
[95]	valid_0's multi_error: 0.124654
[100]	valid_0's multi_error: 0.124654
[105]	valid_0's multi_error: 0.121884
Early stopping, best iteration is:
[8]	valid_0's multi_error: 0.099723
Training until validation scores don't improve for 100 rounds.
[5]	valid_0's multi_error

In [16]:
result_df.to_csv('song1_result.csv', index=False)